In [33]:
import pandas as pd
import datetime
import re
import numpy as np

In [12]:
df = pd.read_excel("../data/france_dataset.xlsx")
cols = df.columns

In [13]:
df['date_hospital_admi'] = pd.to_datetime(df['date_hospital_admi'])
df['date_ICU_admi'] = pd.to_datetime(df['date_ICU_admi'])
df['date_hospital_end'] = pd.to_datetime(df['date_hospital_end'], errors='coerce')
df['date_death'] = pd.to_datetime(df['date_death'], errors='coerce')

In [14]:
df_base = df[['id', 'date_hospital_admi', 'date_hospital_end', 'date_death', 'ICU_DC']]
df_base['end-admi'] = (df_base['date_hospital_end'] - df_base['date_hospital_admi']).dt.days

<ipython-input-14-4c95223f4acf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_base['end-admi'] = (df_base['date_hospital_end'] - df_base['date_hospital_admi']).dt.days


In [15]:
# Admission data: HR_min, HR_max, SBP_min, SBP_max, DBP_min, DBP_max, SAPSII, RR, GCS
df_admi = df.filter([ 'HR_min', 'HR_max', 'SBP_min', 'SBP_max', 'DBP_min', 'DBP_max', 'SAPSII', 'RR', 'GCS'])

In [16]:
all_cols = df.columns.to_list()
for e in ['HR_min', 'HR_max', 'SBP_min', 'SBP_max', 'DBP_min', 'DBP_max', 'SAPSII', 'RR', 'GCS']:
    all_cols.remove(e)
for e in ['id', 'date_hospital_admi', 'date_hospital_end', 'date_death', 'ICU_DC']:
    all_cols.remove(e)
det_cols = all_cols

In [17]:
df_dete = df.iloc[:,23:]

In [34]:
df.dropna(subset=['ICU_DC'])

KeyError: ['ICU_DC']

# kolumny czasowo-śmierciowe

In [39]:
data = df
df =data[['date_hospital_admi','date_ICU_admi','date_death', 'Death_D7', 'Death_D28']][data['date_death'] != np.datetime64('NaT')]

In [40]:
df['date_hospital_admi'] = pd.to_datetime(df['date_hospital_admi'])
df['date_ICU_admi'] = pd.to_datetime(df['date_ICU_admi'])
df['date_death'] = pd.to_datetime(df['date_death'])

In [41]:
df["from_door_to_death"] = df['date_death'] - df['date_hospital_admi']

In [42]:
df

,date_hospital_admi,date_ICU_admi,date_death,Death_D7,Death_D28,from_door_to_death
0,2020-03-26,2020-03-26,NaT,0,0,NaT
1,2020-03-25,2020-03-25,2020-04-01,1,1,7 days
2,2020-03-26,2020-03-27,NaT,0,1,NaT
3,2020-03-21,2020-03-25,NaT,0,1,NaT
4,2020-03-23,2020-03-24,NaT,0,1,NaT
...,...,...,...,...,...,...
173,2020-03-25,2020-03-25,NaT,0,0,NaT
174,2020-03-20,2020-03-27,NaT,0,0,NaT
175,2020-03-25,2020-03-26,NaT,0,0,NaT
176,2020-03-22,2020-03-24,NaT,0,0,NaT


# Martwi bez daty śmierci - jeszcze ok

In [27]:
dead = (data["Death_D7"] != 0) | (data["Death_D28"] != 0)
incorrect = data[(data["date_death"] == np.datetime64()) & dead].filter(["date_death","Death_D7", "Death_D28" ])
incorrect

,date_death,Death_D7,Death_D28


In [28]:
incorrect.count()

date_death    0
Death_D7      0
Death_D28     0
dtype: int64

# Martwi z datą śmierci, ale bez flagi śmieci

In [43]:
not_dead = (df["Death_D7"] == 0) & (df["Death_D28"] == 0)
incorrect2 = df[(df["date_death"] != np.datetime64('NaT')) & not_dead].filter(["date_death","Death_D7", "Death_D28", "from_door_to_death" ])
incorrect2

,date_death,Death_D7,Death_D28,from_door_to_death
0,NaT,0,0,NaT
5,NaT,0,0,NaT
6,NaT,0,0,NaT
9,NaT,0,0,NaT
10,NaT,0,0,NaT
...,...,...,...,...
173,NaT,0,0,NaT
174,NaT,0,0,NaT
175,NaT,0,0,NaT
176,NaT,0,0,NaT


In [44]:
incorrect2.count()

date_death             43
Death_D7              117
Death_D28             117
from_door_to_death     43
dtype: int64

# dodajmy ramy czasowe 

In [45]:
inc2_range = incorrect2[incorrect2["from_door_to_death"] < datetime.timedelta(days=28)]
inc2_range

,date_death,Death_D7,Death_D28,from_door_to_death
22,2020-03-24,0,0,11 days
30,2020-04-07,0,0,7 days
32,2020-04-05,0,0,11 days
35,2020-04-22,0,0,16 days
42,2020-02-14,0,0,16 days
44,2020-03-24,0,0,23 days
46,2020-03-19,0,0,12 days
48,2020-03-29,0,0,24 days
49,2020-04-01,0,0,26 days
56,2020-03-30,0,0,23 days


In [46]:
inc2_range.count()

date_death            38
Death_D7              38
Death_D28             38
from_door_to_death    38
dtype: int64